In [1]:
# basics
import pandas as pd
import numpy as np
import time

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch_directml

# testing
from functions import simulate_game, pick_dice, pick_score
from classes import ScoreSheet, TurnData

In [2]:
gpu = torch_directml.device()

In [3]:
use_gpu = False

## Data Prep

In [4]:
training_df = pd.read_csv('data/training_df.csv')

In [5]:
training_df.columns

Index(['game', 'turn', 'pre_total_score', 'chosen_score_type', 'turn_score',
       'post_total_score', 'chance_score', 'ones_score', 'twos_score',
       'threes_score', 'fours_score', 'fives_score', 'sixes_score',
       'three_kind_score', 'four_kind_score', 'full_house_score',
       'small_straight_score', 'large_straight_score', 'yahtzee_score',
       'chance_potential', 'ones_potential', 'twos_potential',
       'threes_potential', 'fours_potential', 'fives_potential',
       'sixes_potential', 'three_kind_potential', 'four_kind_potential',
       'full_house_potential', 'small_straight_potential',
       'large_straight_potential', 'yahtzee_potential', 'hand_1_dice_1',
       'hand_1_dice_2', 'hand_1_dice_3', 'hand_1_dice_4', 'hand_1_dice_5',
       'picks_1_dice_1', 'picks_1_dice_2', 'picks_1_dice_3', 'picks_1_dice_4',
       'picks_1_dice_5', 'hand_2_dice_1', 'hand_2_dice_2', 'hand_2_dice_3',
       'hand_2_dice_4', 'hand_2_dice_5', 'picks_2_dice_1', 'picks_2_dice_2',
      

In [6]:
X_cols = [
    x for x in training_df.columns if
    x != 'chosen_score_type' and
    x != 'turn_score' and
    x != 'post_total_score'
]

y1_cols = [x for x in training_df.columns if 'picks_1' in x]
y2_cols = [x for x in training_df.columns if 'picks_2' in x]
y3_cols = [x for x in training_df.columns if x == 'chosen_score_type']

X = training_df[[x for x in X_cols if '_potential' not in x] + [x for x in X_cols if '_potential' in x]]
X = X.fillna(-1)

y1 = training_df[y1_cols]
y2 = training_df[y2_cols]
y3 = pd.get_dummies(training_df[y3_cols]).astype(int)

In [7]:
X.columns

Index(['game', 'turn', 'pre_total_score', 'chance_score', 'ones_score',
       'twos_score', 'threes_score', 'fours_score', 'fives_score',
       'sixes_score', 'three_kind_score', 'four_kind_score',
       'full_house_score', 'small_straight_score', 'large_straight_score',
       'yahtzee_score', 'hand_1_dice_1', 'hand_1_dice_2', 'hand_1_dice_3',
       'hand_1_dice_4', 'hand_1_dice_5', 'picks_1_dice_1', 'picks_1_dice_2',
       'picks_1_dice_3', 'picks_1_dice_4', 'picks_1_dice_5', 'hand_2_dice_1',
       'hand_2_dice_2', 'hand_2_dice_3', 'hand_2_dice_4', 'hand_2_dice_5',
       'picks_2_dice_1', 'picks_2_dice_2', 'picks_2_dice_3', 'picks_2_dice_4',
       'picks_2_dice_5', 'hand_3_dice_1', 'hand_3_dice_2', 'hand_3_dice_3',
       'hand_3_dice_4', 'hand_3_dice_5', 'chance_potential', 'ones_potential',
       'twos_potential', 'threes_potential', 'fours_potential',
       'fives_potential', 'sixes_potential', 'three_kind_potential',
       'four_kind_potential', 'full_house_potential',

In [8]:
indices_branch1 = [x for x in range(21)]
indices_branch2 = [x for x in range(31)]
total_features = X.shape[1]

mask1 = torch.zeros(total_features, dtype=torch.bool)
mask1[indices_branch1] = True

mask2 = torch.zeros(total_features, dtype=torch.bool)
mask2[indices_branch2] = True

masks = [mask1, mask2]

## Model Class

In [9]:
class Botzee(nn.Module):
    def __init__(self, input_sizes, lstm_sizes, dice_output_size, score_output_size):
        super(Botzee, self).__init__()
        # branch 1 - binary dice pick 1
        self.lstm1 = nn.LSTM(input_sizes[0], lstm_sizes[0])
        self.branch1 = nn.Linear(lstm_sizes[0], dice_output_size)

        # branch 2 - binary dice pick 2
        self.lstm2 = nn.LSTM(input_sizes[1], lstm_sizes[1])
        self.branch2 = nn.Linear(lstm_sizes[1], dice_output_size)

        # branch 3 - multilabel score pick
        self.lstm3 = nn.LSTM(input_sizes[2], lstm_sizes[2])
        self.branch3 = nn.Linear(lstm_sizes[2], score_output_size)

    def forward(self, x, masks):
        # in case of 2D input, add a batch dimension
        if x.dim() == 1:
            x = x.unsqueeze(0)
        
        # branch 1 - binary
        out1, _ = self.lstm1(x[:, masks[0]])
        out1 = self.branch1(out1)
        dice1_output = torch.sigmoid(out1)

        # branch 2 - binary
        out2, _ = self.lstm2(x[:, masks[1]])
        out2 = self.branch2(out2)
        dice2_output = torch.sigmoid(out2)

        # branch 3 - multilabel
        out3, _ = self.lstm3(x)
        out3 = self.branch3(out3)
        score_output = F.softmax(out3, dim = 1)

        return dice1_output, dice2_output, score_output

In [10]:
class BotzeeDataset(Dataset):
    def __init__(self, features, targets_branch1, targets_branch2, targets_branch3):
        self.features = features
        self.targets_branch1 = targets_branch1
        self.targets_branch2 = targets_branch2
        self.targets_branch3 = targets_branch3

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Convert to tensors
        x = torch.tensor(self.features.iloc[idx, :].values, dtype=torch.float32)
        y1 = torch.tensor(self.targets_branch1.iloc[idx, :].values, dtype=torch.float32)
        y2 = torch.tensor(self.targets_branch2.iloc[idx, :].values, dtype=torch.float32)
        y3 = torch.tensor(self.targets_branch3.iloc[idx, :].values, dtype = torch.float32)  # Assuming single value for branch 3
        return x, (y1, y2, y3)

In [11]:
# model params
input_sizes = [
    len([x for x in masks[0] if x == True]),
    len([x for x in masks[1] if x == True]),
    X.shape[1]
]
lstm_sizes = [32, 32, 32]

# instantiate model
model = Botzee(input_sizes = input_sizes, lstm_sizes = lstm_sizes, dice_output_size = 5, score_output_size = 13)
if use_gpu:
    model = model.to(gpu)

# create the dataset and define DataLoader
dataset = BotzeeDataset(X, y1, y2, y3)
dataloader = DataLoader(dataset, batch_size = 13, shuffle = False)

# choose optimizer and loss functions
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fn1 = nn.BCELoss()  # Binary Cross Entropy for first two branches
loss_fn2 = nn.CrossEntropyLoss()  # Cross Entropy for third branch

# choose number of epochs
n_epochs = 1

In [12]:
start_time = time.perf_counter()

for epoch in range(1,n_epochs+1):

    for X_batch, Y_batch in dataloader:

        # move to GPU
        if use_gpu:
            X_batch = X_batch.to(gpu)
            Y_batch = [y.to(gpu) for y in Y_batch]

        # Forward pass
        out1, out2, out3 = model.forward(X_batch, masks)

        # Calculate loss for each branch
        loss1 = loss_fn1(out1, Y_batch[0])
        loss2 = loss_fn1(out2, Y_batch[1])
        loss3 = loss_fn2(out3, Y_batch[2])

        # Total loss
        total_loss = loss1 + loss2 + loss3

        # Backward pass and optimize
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        elapsed_time = time.perf_counter() - start_time
        print(f'Epoch [{epoch}] - Loss: {total_loss.item():.4f} - Time elapsed: {elapsed_time/60:.2f} min')

In [13]:
# create data for prediction
y_pred = X_batch[0:5,:].clone().detach()
# y_pred = y_pred.unsqueeze(0)
if use_gpu:
    y_pred.to(gpu)

# switch model to eval mode
model.eval()

# make predictions
with torch.no_grad():
    predictions = model(y_pred, masks)

dice_picks_1 = []
for pick in predictions[0][0]:
    if pick >= 0.5:
        dice_picks_1.append(1)
    else:
        dice_picks_1.append(0)

dice_picks_2 = []
for pick in predictions[1][0]:
    if pick >= 0.5:
        dice_picks_2.append(1)
    else:
        dice_picks_2.append(0)

score_pick = predictions[2][0].argmax()

print(
    f'Picks after first roll: {dice_picks_1}\n',
    f'Picks after second roll: {dice_picks_2}\n',
    f'Score Pick: {score_pick}'
)

Picks after first roll: [0, 1, 1, 1, 0]
 Picks after second roll: [1, 1, 1, 1, 1]
 Score Pick: 12


In [14]:
def model_pick_dice(hand, model, round, inputs, masks):
    model = model.eval()
    with torch.no_grad():
        predictions = model(inputs, masks)
    dice_picks = []
    for pick in predictions[round-1][0]:
        if pick >= 0.5:
            dice_picks.append(1)
        else:
            dice_picks.append(0)
    return dice_picks

In [15]:
def model_pick_score(potential_scores, model, inputs, masks):
    model = model.eval()
    with torch.no_grad():
        predictions = model(inputs, masks)
    score_pick = predictions[2][0].argmax()
    return score_pick

In [16]:
inputs = X_batch[0,:].clone().detach()

hand = np.sort(np.random.randint(1, 7, 5))[::-1]
print(hand)

dice_picks, keepers = pick_dice(hand, model_pick_dice, model = model, round = 1, inputs = inputs, masks = masks)

print(dice_picks, keepers)

[6 4 2 1 1]
[0, 1, 1, 1, 0] [4 2 1]


In [44]:
# def play_botzee(game_number, model, masks):
game_number = 1
        
game_number = game_number

# to capture game data
game_data = []

# instantiate a new score sheet and game variables
score_sheet = ScoreSheet()
score_sheet.initialize_score_types()

# 13 rounds per game
for turn_number in range(1):

    # instantiate a new game data object
    remaining_score_types = [x[0] for x in score_sheet.scores.items() if x[1] == None]
    turn = TurnData(game_number, turn_number, score_sheet.total)
    for score in score_sheet.get_current_scores().items():
        turn.__dict__.update([score])

    # first roll
    hand = np.sort(np.random.randint(1, 7, 5))[::-1]
    for idx, die in enumerate(hand):
        turn.__dict__.update([(f'hand_1_dice_{idx+1}', die)])

    # prep data for model
    # inputs.update(score_sheet.get_current_scores())


    # # choose dice to keep
    # turn.dice_picks_1, hand_1_keepers = pick_dice(turn.hand_1, dice_decision_function)

    # # second roll
    # roll = np.random.randint(1, 7, 5 - len(hand_1_keepers))
    # turn.hand_2 = np.sort(np.append(hand_1_keepers, roll))[::-1]

    # # choose dice to keep
    # turn.dice_picks_2, hand_2_keepers = pick_dice(turn.hand_2, dice_decision_function)

    # # third roll
    # roll = np.random.randint(1, 7, 5 - len(hand_2_keepers))
    # turn.hand_3 = np.sort(np.append(hand_2_keepers, roll))[::-1]

    # # loop through all remaining scores, check which are eligible and calculate potential score
    # potential_scores = {}
    # for score_type, score in zip(score_sheet.score_types, score_sheet.scores.items()):
    #     if score_type.check_condition(turn.hand_3) and score[1] == None:
    #         value = score_type.calculate_score(turn.hand_3)
    #     elif score[1] != None:
    #         value = -1
    #     else:
    #         value = 0
    #     potential_scores[score_type.name] = value

    # # choose score type
    # turn.chosen_score_type, turn.turn_score = pick_score(potential_scores, score_decision_function)

    # # mark score
    # score_sheet.mark_score(turn.chosen_score_type, turn.turn_score)
    # turn.post_total_score = score_sheet.total

    # # capture data
    # turn_data = turn.capture_data()

    # remaining_score_types = dict(zip(
    #     [f'{x}_score' for x in score_sheet.scores.keys()],
    #     score_sheet.scores.values()
    # ))
    # turn_data.update(remaining_score_types)

    # for old_key in list(potential_scores.keys()):
    #     potential_scores[f'{old_key}_potential'] = potential_scores.pop(old_key)
    # turn_data.update(potential_scores)

    # game_data.append(turn_data)

# return game_data

In [45]:
turn.capture_data()

{'game': 1,
 'turn': 0,
 'pre_total_score': 0,
 'chance_score': -1,
 'ones_score': -1,
 'twos_score': -1,
 'threes_score': -1,
 'fours_score': -1,
 'fives_score': -1,
 'sixes_score': -1,
 'three_kind_score': -1,
 'four_kind_score': -1,
 'full_house_score': -1,
 'small_straight_score': -1,
 'large_straight_score': -1,
 'yahtzee_score': -1,
 'hand_1_dice_1': 6,
 'hand_1_dice_2': 3,
 'hand_1_dice_3': 1,
 'hand_1_dice_4': 1,
 'hand_1_dice_5': 1}

In [40]:
dict_.update([pair])

In [41]:
dict_

{'chance_score': -1,
 'ones_score': -1,
 'twos_score': -1,
 'threes_score': -1,
 'fours_score': -1,
 'fives_score': -1,
 'sixes_score': -1,
 'three_kind_score': -1,
 'four_kind_score': -1,
 'full_house_score': -1,
 'small_straight_score': -1,
 'large_straight_score': -1,
 'yahtzee_score': -1,
 'hand_1_dice_0': 6}

In [18]:
X.columns

Index(['game', 'turn', 'pre_total_score', 'chance_score', 'ones_score',
       'twos_score', 'threes_score', 'fours_score', 'fives_score',
       'sixes_score', 'three_kind_score', 'four_kind_score',
       'full_house_score', 'small_straight_score', 'large_straight_score',
       'yahtzee_score', 'hand_1_dice_1', 'hand_1_dice_2', 'hand_1_dice_3',
       'hand_1_dice_4', 'hand_1_dice_5', 'picks_1_dice_1', 'picks_1_dice_2',
       'picks_1_dice_3', 'picks_1_dice_4', 'picks_1_dice_5', 'hand_2_dice_1',
       'hand_2_dice_2', 'hand_2_dice_3', 'hand_2_dice_4', 'hand_2_dice_5',
       'picks_2_dice_1', 'picks_2_dice_2', 'picks_2_dice_3', 'picks_2_dice_4',
       'picks_2_dice_5', 'hand_3_dice_1', 'hand_3_dice_2', 'hand_3_dice_3',
       'hand_3_dice_4', 'hand_3_dice_5', 'chance_potential', 'ones_potential',
       'twos_potential', 'threes_potential', 'fours_potential',
       'fives_potential', 'sixes_potential', 'three_kind_potential',
       'four_kind_potential', 'full_house_potential',

In [19]:
X.iloc[3,:]

game                         1.0
turn                         4.0
pre_total_score             67.0
chance_score                23.0
ones_score                  -1.0
twos_score                  -1.0
threes_score                 6.0
fours_score                 -1.0
fives_score                 -1.0
sixes_score                 -1.0
three_kind_score            19.0
four_kind_score             -1.0
full_house_score            25.0
small_straight_score        -1.0
large_straight_score        -1.0
yahtzee_score               -1.0
hand_1_dice_1                6.0
hand_1_dice_2                5.0
hand_1_dice_3                3.0
hand_1_dice_4                3.0
hand_1_dice_5                2.0
picks_1_dice_1               0.0
picks_1_dice_2               0.0
picks_1_dice_3               1.0
picks_1_dice_4               1.0
picks_1_dice_5               0.0
hand_2_dice_1                6.0
hand_2_dice_2                3.0
hand_2_dice_3                3.0
hand_2_dice_4                2.0
hand_2_dic